In [2]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('RAA_Naver_많이본뉴스기사제목.csv')

## 전처리
    1. 괄호 및 괄호 안 문자 제거 ex <뉴욕증시>, [시황]
    2. 특수문자 제거(한글,영어,- 제외하고 나머지)

In [14]:
import re
import konlpy

from konlpy.tag import Mecab

In [15]:
Bracket = re.compile(r'(\([^)]*\))|(\<[^)]*\>)|(\[[^)]*\])|(\{[^)]*\})')
df['pp뉴스제목명'] = df.뉴스제목명.map(lambda x:re.sub(Bracket,'',x))

In [16]:
def delword(x):
    word = re.compile(r'[^ㄱ-ㅎ| 가-힣|A-Z|a-z|\-]')
    x = word.sub(' ',x)
    x = re.sub(r' +',' ',x)
    return x

df['pp뉴스제목명'] = df.pp뉴스제목명.map(lambda x:delword(x))

In [17]:
df

,기준일자,뉴스제목명,뉴스URL,pp뉴스제목명
0,20170101,SK이노베이션 “올해 3조원 규모 투자”,https://finance.naver.com/news/news_read.nhn?a...,SK이노베이션 올해 조원 규모 투자
1,20170101,“상호금융 출자금은 원금 보장 안돼요”,https://finance.naver.com/news/news_read.nhn?a...,상호금융 출자금은 원금 보장 안돼요
2,20170101,경제부처 수장들 고사성어로 위기 극복 강조,https://finance.naver.com/news/news_read.nhn?a...,경제부처 수장들 고사성어로 위기 극복 강조
3,20170101,비싼 ‘복어탕’이 대형마트에 등장한 이유는?,https://finance.naver.com/news/news_read.nhn?a...,비싼 복어탕 이 대형마트에 등장한 이유는
4,20170101,수출 2년 연속 감소…교역액 9천억달러 턱걸이,https://finance.naver.com/news/news_read.nhn?a...,수출 년 연속 감소 교역액 천억달러 턱걸이
...,...,...,...,...
2752627,20210726,"외벌이는 소득 8605만원, 맞벌이는 1억532만원까지 지원금 받는다",https://finance.naver.com/news/news_read.nhn?a...,외벌이는 소득 만원 맞벌이는 억 만원까지 지원금 받는다
2752628,20210726,"카카오그룹, 국내 5번째 시총 100조 전망",https://finance.naver.com/news/news_read.nhn?a...,카카오그룹 국내 번째 시총 조 전망
2752629,20210726,카카오뱅크 공모주 오늘부터 청약···1주라도 더 받으려면?,https://finance.naver.com/news/news_read.nhn?a...,카카오뱅크 공모주 오늘부터 청약 주라도 더 받으려면
2752630,20210726,카카오뱅크 오늘부터 공모청약…1주라도 더 받으려면?,https://finance.naver.com/news/news_read.nhn?a...,카카오뱅크 오늘부터 공모청약 주라도 더 받으려면


## Custom Pos
    1. Mecab pos 문제점
        a. 한개의 단어를 나눔 ex >> 카카오뱅크 : ('카카오', 'NNP'), ('뱅크', 'NNG')
                                    SK이노베이션 : ('SK', 'SL'), ('이노베이션', 'NNG')
                                    
           - Wanted : 명사 or 외국어 태그가 연속으로 이어질 때 NNP로 묶은 결과도 노출
               ex >> SK이노베이션 >> ('SK이노베이션','NNP'), ('SK','SL'), ('이노베이션', 'NNG')
                     카카오뱅크   >> ('카카오뱅크','NNP'), ('카카오', 'NNP'), ('뱅크', 'NNG')
        
        b. 1개의 단어로 이루어져 있을 경우 해석이 애매함 ex >> 것, 수 ..
            - Threshold(min_lenn) 기준으로 적으면 제외
    
    2. 위의 처리를 하고 나온 결과물의 명사 수가 너무 적을 경우 뉴스 기사 제외

In [18]:
mecab = Mecab('C:/mecab/mecab-ko-dic')

In [19]:
def custompos(sent,min_len = 1):
    sents = sent.split(" ")
    tags = ['NNP','SL','NNG']
    result = []
    for se in sents:
        tagging = mecab.pos(se)
        if len(tagging) == 1:
            v,tg = tagging[0]
            if tg in tags and len(v) > min_len:
                result.append(v)
        else:
            tmp = ''
            flag = False
            for v,tg in tagging:
                if tmp == '':
                    if tg in tags and len(v) > min_len:
                        tmp = v
                        result.append(v)
                else:
                    if tg in tags and len(v) > min_len:
                        tmp += v
                        result.append(v)
                        flag = True
                    elif flag:
                        result.append(tmp)
                        tmp = ''
                        flag = False
            if flag:
                result.append(tmp)
    return result

In [20]:
min_count = 3
df['nouns'] = df.pp뉴스제목명.map(lambda x: ' '.join(custompos(x)))
df = df[df.nouns.apply(lambda x: True if len(x.split(' ')) > min_count  else False)].reset_index(drop = True)

In [21]:
df.groupby('기준일자').count().describe()

,뉴스제목명,뉴스URL,pp뉴스제목명,nouns
count,1668.000000,1668.000000,1668.000000,1668.000000
mean,1514.917866,1514.917866,1514.917866,1514.917866
std,946.205353,946.205353,946.205353,946.205353
min,16.000000,16.000000,16.000000,16.000000
25%,563.750000,563.750000,563.750000,563.750000
50%,1825.000000,1825.000000,1825.000000,1825.000000
75%,2272.750000,2272.750000,2272.750000,2272.750000
max,4398.000000,4398.000000,4398.000000,4398.000000


## 분석
    1. 목표
    
        하루 평균 1500건이 넘는 뉴스 중에서 비슷한 내용의 기사들을 그룹화하고 그중 가장 중심성이 높은 기사 한 건을 노출
        
    2. 방법론 : Graph-based News Representation
    
        a. 가정 
            - 많은 언론사들이 유사한 제목의 비슷한 기사를 보도한다면 해당 사건이나 이슈는 사회에 파급력이 클 가능성이 높다.
            
        b. 방법
            - Degree centrality(간선중심성) : 기사 그룹과 중심성 Index 가중치로 활용
                BFS를 활용한 기사간 연결확인 >> Threshold : 몇 개 이상의 단어가 겹쳐야 연결된 것인지
            
            - Eigenvector centrality(고유벡터중심성)
                기사간 연결을 나타낸 행렬을 PCA >> First Eigenvctor만 활용

    3. 적용
        a. 날짜별로(yyyymmdd) 뉴스 기사 그룹, 간선중심성, 고유백터중심성 계산
        b. 기사의 중심성 Index = 간선중심성 * 고유백터중심성

In [12]:
df = df[df.뉴스제목명.map(lambda x: True if ('[표]' not in x) and ('[사진]' not in x) else False)].reset_index(drop = True)

In [13]:
from collections import deque
from sklearn import decomposition
from tqdm import tqdm

def ftn(yyyymmdd,Threshold):
    tmp = df[df.기준일자 == yyyymmdd].loc[:,'nouns'].to_list()
    dic = {idx:set(x.split(" ")) for idx,x in enumerate(tmp)}
    Len = len(dic)
    
    v,pc = [[0] * Len for _ in range(Len)],[[0] * Len for _ in range(Len)]
    
    for st in range(Len):
        for en in range(st+1,Len):
            inter = dic[st].intersection(dic[en])
            v[st][en],v[en][st] = len(inter),len(inter)
            pc[st][en],pc[en][st] = 1,1
    
    group = [0] * Len
    
    group_cnt = 1
    for idx,g in enumerate(group):
        if g == 0:
            que = deque([idx])
            while que:
                ix = que.popleft()
                if group[ix] == 0:
                    group[ix] = group_cnt
                    que.extend([i for i,v in enumerate(v[ix]) if v >= Threshold])
            group_cnt += 1
    
    pca = decomposition.PCA(n_components=1)
    pca_result = pca.fit(pc)
    
    return group,pca_result.components_[0],np.sum(v,axis = 0)

In [14]:
group,components,cnt = [],[],[]
for ymd in tqdm(df.기준일자.drop_duplicates().to_list()):
    a,b,c = ftn(ymd,4)
    group.extend(a)
    components.extend(b)
    cnt.extend(c)

100%|██████████████████████████████████████████████████████████████████████████████| 1668/1668 [45:43<00:00,  1.64s/it]


In [15]:
df['group'] = group
df['components'] = components
df['cnt'] = cnt

In [18]:
df['idx'] = df.apply(lambda x: abs(x['components']) * x['cnt'],axis = 1)

In [22]:
df

,기준일자,뉴스제목명,뉴스URL,pp뉴스제목명,nouns,group,components,cnt,idx
0,20170101,"SK이노베이션, 공격 투자로 기업가치 혁신 나선다",https://finance.naver.com/news/news_read.nhn?a...,SK이노베이션 공격 투자로 기업가치 혁신 나선다,SK 이노베이션 SK이노베이션 공격 투자 기업 가치 기업가치 혁신,1,-0.093580,122,11.416782
1,20170101,"김준 SK이노베이션 사장 ""올해 3조 투자...채용 확대""",https://finance.naver.com/news/news_read.nhn?a...,김준 SK이노베이션 사장 올해 3조 투자 채용 확대,김준 SK 이노베이션 SK이노베이션 사장 올해 투자 채용 확대,1,0.072543,139,10.083527
2,20170101,"SK이노베이션, 3조원 투자 '공격 경영'…5년간 1200명 채용",https://finance.naver.com/news/news_read.nhn?a...,SK이노베이션 3조원 투자 공격 경영 5년간 1200명 채용,SK 이노베이션 SK이노베이션 조원 투자 공격 경영 채용,1,-0.055493,100,5.549299
3,20170101,"SK이노베이션, 새해 3조원 투자 선언",https://finance.naver.com/news/news_read.nhn?a...,SK이노베이션 새해 3조원 투자 선언,SK 이노베이션 SK이노베이션 새해 조원 투자 선언,1,0.035322,149,5.262949
4,20170101,SK이노베이션 “올해 3조원 규모 투자”,https://finance.naver.com/news/news_read.nhn?a...,SK이노베이션 올해 3조원 규모 투자,SK 이노베이션 SK이노베이션 올해 조원 규모 투자,1,-0.044142,117,5.164609
...,...,...,...,...,...,...,...,...,...
2481794,20210726,"외벌이는 소득 8605만원, 맞벌이는 1억532만원까지 지원금 받는다",https://finance.naver.com/news/news_read.nhn?a...,외벌이는 소득 8605만원 맞벌이는 1억532만원까지 지원금 받는다,외벌 소득 맞벌이 지원금,12,0.092987,0,0.000000
2481795,20210726,"카카오그룹, 국내 5번째 시총 100조 전망",https://finance.naver.com/news/news_read.nhn?a...,카카오그룹 국내 5번째 시총 100조 전망,카카오 그룹 카카오그룹 국내 전망,13,0.114660,4,0.458639
2481796,20210726,카카오뱅크 오늘부터 공모청약…1주라도 더 받으려면?,https://finance.naver.com/news/news_read.nhn?a...,카카오뱅크 오늘부터 공모청약 1주라도 더 받으려면,카카오 뱅크 카카오뱅크 오늘 공모 청약 공모청약,14,0.158005,7,1.106036
2481797,20210726,카카오뱅크 공모주 오늘부터 청약···1주라도 더 받으려면?,https://finance.naver.com/news/news_read.nhn?a...,카카오뱅크 공모주 오늘부터 청약 1주라도 더 받으려면,카카오 뱅크 카카오뱅크 공모주 오늘 청약,14,0.136332,8,1.090660


In [20]:
df = df.sort_values(['기준일자','group','idx'],ascending = [True,True,False]).reset_index(drop = True)

In [32]:
df.iloc[:,[0,3,1,2,4,5,6]].to_csv('분석.csv',
         index = False)

In [33]:
df

,기준일자,뉴스제목명,nouns,group,components,cnt,idx
0,20170101,"SK이노베이션, 공격 투자로 기업가치 혁신 나선다",SK 이노베이션 SK이노베이션 공격 투자 기업 가치 기업가치 혁신,1,-0.093580,122,11.416782
1,20170101,"김준 SK이노베이션 사장 ""올해 3조 투자...채용 확대""",김준 SK 이노베이션 SK이노베이션 사장 올해 투자 채용 확대,1,0.072543,139,10.083527
2,20170101,"SK이노베이션, 3조원 투자 '공격 경영'…5년간 1200명 채용",SK 이노베이션 SK이노베이션 조원 투자 공격 경영 채용,1,-0.055493,100,5.549299
...,...,...,...,...,...,...,...
2481796,20210726,카카오뱅크 오늘부터 공모청약…1주라도 더 받으려면?,카카오 뱅크 카카오뱅크 오늘 공모 청약 공모청약,14,0.158005,7,1.106036
2481797,20210726,카카오뱅크 공모주 오늘부터 청약···1주라도 더 받으려면?,카카오 뱅크 카카오뱅크 공모주 오늘 청약,14,0.136332,8,1.090660
2481798,20210726,탄소절감의 역설···알루미늄이 뜬다,탄소 절감 탄소절감 역설 알루미늄,15,0.179678,0,0.000000


In [3]:
df = pd.read_csv('분석.csv')

In [10]:
df = pd.merge(df,
              df.loc[:,['기준일자','group','뉴스제목명']].groupby(['기준일자','group'],as_index= False).count().rename(columns = {
                  '뉴스제목명' : 'Group_count'
              }),
              on = ['기준일자','group'])

In [21]:
df

,기준일자,group,뉴스제목명,nouns,components,cnt,idx,Group_count
0,20170101,1,"SK이노베이션, 공격 투자로 기업가치 혁신 나선다",SK 이노베이션 SK이노베이션 공격 투자 기업 가치 기업가치 혁신,-0.093580,122,11.416782,14
1,20170101,1,"김준 SK이노베이션 사장 ""올해 3조 투자...채용 확대""",김준 SK 이노베이션 SK이노베이션 사장 올해 투자 채용 확대,0.072543,139,10.083527,14
2,20170101,1,"SK이노베이션, 3조원 투자 '공격 경영'…5년간 1200명 채용",SK 이노베이션 SK이노베이션 조원 투자 공격 경영 채용,-0.055493,100,5.549299,14
3,20170101,1,"SK이노베이션, 새해 3조원 투자 선언",SK 이노베이션 SK이노베이션 새해 조원 투자 선언,0.035322,149,5.262949,14
4,20170101,1,SK이노베이션 “올해 3조원 규모 투자”,SK 이노베이션 SK이노베이션 올해 조원 규모 투자,-0.044142,117,5.164609,14
...,...,...,...,...,...,...,...,...
2481794,20210726,12,"외벌이는 소득 8605만원, 맞벌이는 1억532만원까지 지원금 받는다",외벌 소득 맞벌이 지원금,0.092987,0,0.000000,1
2481795,20210726,13,"카카오그룹, 국내 5번째 시총 100조 전망",카카오 그룹 카카오그룹 국내 전망,0.114660,4,0.458639,1
2481796,20210726,14,카카오뱅크 오늘부터 공모청약…1주라도 더 받으려면?,카카오 뱅크 카카오뱅크 오늘 공모 청약 공모청약,0.158005,7,1.106036,2
2481797,20210726,14,카카오뱅크 공모주 오늘부터 청약···1주라도 더 받으려면?,카카오 뱅크 카카오뱅크 공모주 오늘 청약,0.136332,8,1.090660,2


In [27]:
df

,기준일자,group,뉴스제목명,nouns,components,cnt,idx,Group_count
0,20170101,1,"SK이노베이션, 공격 투자로 기업가치 혁신 나선다",SK 이노베이션 SK이노베이션 공격 투자 기업 가치 기업가치 혁신,-0.093580,122,11.416782,14
1,20170101,1,"김준 SK이노베이션 사장 ""올해 3조 투자...채용 확대""",김준 SK 이노베이션 SK이노베이션 사장 올해 투자 채용 확대,0.072543,139,10.083527,14
2,20170101,1,"SK이노베이션, 3조원 투자 '공격 경영'…5년간 1200명 채용",SK 이노베이션 SK이노베이션 조원 투자 공격 경영 채용,-0.055493,100,5.549299,14
3,20170101,1,"SK이노베이션, 새해 3조원 투자 선언",SK 이노베이션 SK이노베이션 새해 조원 투자 선언,0.035322,149,5.262949,14
4,20170101,1,SK이노베이션 “올해 3조원 규모 투자”,SK 이노베이션 SK이노베이션 올해 조원 규모 투자,-0.044142,117,5.164609,14
...,...,...,...,...,...,...,...,...
2481794,20210726,12,"외벌이는 소득 8605만원, 맞벌이는 1억532만원까지 지원금 받는다",외벌 소득 맞벌이 지원금,0.092987,0,0.000000,1
2481795,20210726,13,"카카오그룹, 국내 5번째 시총 100조 전망",카카오 그룹 카카오그룹 국내 전망,0.114660,4,0.458639,1
2481796,20210726,14,카카오뱅크 오늘부터 공모청약…1주라도 더 받으려면?,카카오 뱅크 카카오뱅크 오늘 공모 청약 공모청약,0.158005,7,1.106036,2
2481797,20210726,14,카카오뱅크 공모주 오늘부터 청약···1주라도 더 받으려면?,카카오 뱅크 카카오뱅크 공모주 오늘 청약,0.136332,8,1.090660,2


In [92]:
tmp = df.sort_values(['기준일자','Group_count','idx'],ascending=[True,False,False])

In [93]:
tmp

,기준일자,group,뉴스제목명,nouns,components,cnt,idx,Group_count
284,20170101,172,"삼성, ‘플렉스워시’세탁기 ‘플렉스드라이’건조기 CES 2017서 공개",삼성 플렉스 워시 플렉스워시 세탁기 플렉스 드라이 플렉스드라이 건조기 CES 공개,0.114012,118,13.453365,21
285,20170101,172,"삼성, '플렉스워시'와 '플렉스드라이' 건조기 최초 공개",삼성 플렉스 워시 플렉스워시 플렉스 드라이 플렉스드라이 건조기 최초 공개,0.068174,107,7.294630,21
286,20170101,172,"삼성전자, '플렉스워시'·'플렉스드라이'",삼성전자 플렉스 워시 플렉스워시 플렉스 드라이 플렉스드라이,-0.097737,72,7.037044,21
287,20170101,172,"삼성, 복합세탁기·건조기 '플렉스워시·드라이' 선보여",삼성 복합 세탁기 복합세탁기 건조기 플렉스 워시 플렉스워시 드라이,0.067260,87,5.851632,21
288,20170101,172,삼성， CES서 업계 최초 종합세탁솔루션 공개,삼성 CES 업계 최초 종합 세탁 솔루션 종합세탁솔루션 공개,-0.068531,84,5.756579,21
...,...,...,...,...,...,...,...,...
2481788,20210726,6,bhc에 아웃백 넘기는 스카이레이크…엑시트 박차 가한다,bhc 아웃백 스카이 레이크 스카이레이크 엑시트,0.017133,0,0.000000,1
2481790,20210726,8,"네이버, 2분기 호실적 타고 시가총액 3위 굳히나",네이버 분기 호실 시가총액,0.017133,0,0.000000,1
2481793,20210726,11,"생활부터 취업까지, 열여덟 청년들 자립 돕기 온힘",생활 취업 청년 자립,0.071314,0,0.000000,1
2481794,20210726,12,"외벌이는 소득 8605만원, 맞벌이는 1억532만원까지 지원금 받는다",외벌 소득 맞벌이 지원금,0.092987,0,0.000000,1


In [94]:
tmp = tmp.groupby(['기준일자','group']).head(1)

In [97]:
tmp.loc[:,['기준일자','뉴스제목명','idx','Group_count']].to_csv('Results.csv',
         index = False)

In [53]:
tmp.groupby('기준일자').head(15).loc[:,['기준일자','뉴스제목명']].to_csv('sample.csv',
         index = False)

In [45]:
tmp.groupby(['기준일자','group']).head(1).to_csv('sample.csv',
         index = False)

In [65]:
ttmp = tmp.groupby(['기준일자','group']).head(1)

In [66]:
ttmp = ttmp[ttmp.기준일자 == 20170707].loc[:,['기준일자','뉴스제목명','idx','Group_count','group']].head(15)
ttmp.index = range(1,16)
ttmp

,기준일자,뉴스제목명,idx,Group_count,group
1,20170707,"[특징주]삼성전자, 분기 사상 최대 실적에 발표에도 '약세'",45.316870,226,27
2,20170707,"경찰, 대한항공 본사 압수수색…횡령·배임 혐의 수사(속보)",13.740401,48,9
3,20170707,"에스디시스템, 파나마기업과 21억 규모 유지보수 계약 체결",8.785467,32,162
4,20170707,"신한銀, 본부부서 조직 개편…현장 영업동력 강화 및 커뮤니티 지원",11.605676,22,104
5,20170707,"GS, 당진에 LNG복합발전소 준공..국내 최대 민간발전사 우뚝",5.889676,18,4
6,20170707,"KD건설, 179억원 규모 유상증자 결정",3.603000,18,65
7,20170707,뉴욕증시 민간고용 부진 등에 약세…다우 0.74% 하락 마감,11.684119,16,78
8,20170707,"한국씨티은행, 영업점 5곳 오늘 폐점‥이달중 35계 문닫아",5.386825,16,20
9,20170707,"금호타이어 채권단, 지난해 경영평가 D등급 확정(2보)",4.162548,15,34
10,20170707,"삼성자산운용, 유럽시장에 공모펀드 선보인다",6.579900,14,85


In [108]:
pd.set_option("display.max_rows",300)

In [109]:
df[(df.기준일자 == 20170707) & (df.group == 27)].loc[:,['기준일자','group','뉴스제목명']]

,기준일자,group,뉴스제목명
285557,20170707,27,"[특징주]삼성전자, 분기 사상 최대 실적에 발표에도 '약세'"
285558,20170707,27,"삼성전자 사상 최대 영업이익, 모바일 역할 톡톡…갤럭시노트FE 기대↑"
285559,20170707,27,[빅데이터MSI]7일 빅데이터 '핫 키워드'…실적·시장·서비스·영업이익·스마트폰
285560,20170707,27,"LG전자, 'G6' 부진으로 2분기 영업이익 6641억원밖에 못 내"
285561,20170707,27,"LG전자, 2분기 영업이익 6641억원…전년동기보다 13.6%↑"
285562,20170707,27,LG전자 2분기 영업이익 6641억원…기대치 하회(상보)
285563,20170707,27,삼성전자 2분기 영업익 14조원
285564,20170707,27,"영업익 14조… 삼성전자, 애플 넘어 글로벌 1위"
285565,20170707,27,"삼성전자, 반도체 '날다'…2분기 영업익 8조 추정(상보)"
285566,20170707,27,삼성전자 2분기 영업이익률 23.3%…사상 최고치(속보)


In [125]:
df[(df.기준일자 == 20170707) & (df.group == 219)].loc[:,['기준일자','group','뉴스제목명']].head(3)

,기준일자,group,뉴스제목명
286239,20170707,219,"[특징주]대한항공, 본사 압수수색 소식에 하락"
286240,20170707,219,[특징주] 대한항공 압수수색에 한진그룹株 동반 하락(종합)
286241,20170707,219,"[특징주]대한항공, 경찰 압수수색 소식에 하락세 전환"


In [41]:
tmp = df[(df.기준일자 == 20170707) & (df.group == 9)].loc[:,['기준일자','뉴스제목명']].reset_index(drop = True)

In [43]:
tmp.index = range(1,49)

In [44]:
tmp

,기준일자,뉴스제목명
1,20170707,"경찰, 대한항공 본사 압수수색…횡령·배임 혐의 수사(속보)"
2,20170707,"경찰, 대한항공 압수수색…회장 자택공사 관련 비리 혐의(종합)"
3,20170707,"경찰, 대한항공 본사 압수수색…횡령ㆍ배임 수사(속보)"
...,...,...
46,20170707,"대한항공 ""경찰 수사 최대한 협조···자체 진상조사도 진행"""
47,20170707,"경찰,대한항공 본사 압수수색"
48,20170707,"[종합]경찰, 대한항공 본사 압수수색···배임 혐의"
